In [ ]:
# | default_exp _server

In [ ]:
# | export

import asyncio
import multiprocessing
import signal
import threading
from contextlib import contextmanager
from typing import *

import asyncer
import typer

from fastkafka._components.helpers import _import_from_string
from fastkafka._components.logger import get_logger, supress_timestamps

In [ ]:
import os
from time import sleep

from pydantic import BaseModel

from fastkafka._components.test_dependencies import generate_app_in_tmp
from fastkafka.testing import ApacheKafkaBroker

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

supress_timestamps(False)
logger = get_logger(__name__, level=20)

In [ ]:
# supress_timestamps()
# logger = get_logger(__name__, level=20)
logger.info("ok")

23-05-12 13:15:31.504 [INFO] __main__: ok


In [ ]:
# | export


class ServerProcess:
    def __init__(self, app: str, kafka_broker_name: str):
        self.app = app
        self.should_exit = False
        self.kafka_broker_name = kafka_broker_name

    def run(self) -> None:
        return asyncio.run(self._serve())

    async def _serve(self) -> None:
        self._install_signal_handlers()

        self.application = _import_from_string(self.app)
        self.application.set_kafka_broker(self.kafka_broker_name)

        async with self.application:
            await self._main_loop()

    def _install_signal_handlers(self) -> None:
        if threading.current_thread() is not threading.main_thread():
            raise RuntimeError()

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )

        def handle_exit(sig: int) -> None:
            self.should_exit = True

        for sig in HANDLED_SIGNALS:
            loop.add_signal_handler(sig, handle_exit, sig)

    async def _main_loop(self) -> None:
        while not self.should_exit:
            await asyncio.sleep(0.1)

In [ ]:
# | export

_app = typer.Typer()


@_app.command()
def run_fastkafka_server_process(
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
    kafka_broker: str = typer.Option(
        ...,
        help="kafka_broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastaKafka class.",
    ),
) -> None:
    ServerProcess(app, kafka_broker).run()

In [ ]:
# | notest

print("WARNING: make sure you save the notebook before running this cell\n")

print("Exporting and installing the new version of the CLI command...")
await asyncio.create_subprocess_exec("nbdev_export")
export_process = await asyncio.create_subprocess_exec("nbdev_export")
await export_process.wait()
assert export_process.returncode == 0

install_process = await asyncio.create_subprocess_exec(
    "pip", "install", "-e", "..[all]"
)
await install_process.wait()
assert install_process.returncode == 0

print("ok")


Exporting and installing the new version of the CLI command...
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///work/fastkafka
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Attempting uninstall: fastkafka
    Found existing installation: fastkafka 0.6.0rc0
    Uninstalling fastkafka-0.6.0rc0:
      Successfully uninstalled fastkafka-0.6.0rc0
  Running setup.py develop for fastkafka
ok



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
# | export


async def terminate_asyncio_process(p: asyncio.subprocess.Process) -> None:
    logger.info(f"terminate_asyncio_process(): Terminating the process {p.pid}...")
    # Check if SIGINT already propagated to process
    try:
        await asyncio.wait_for(p.wait(), 1)
        logger.info(
            f"terminate_asyncio_process(): Process {p.pid} was already terminated."
        )
        return
    except asyncio.TimeoutError:
        pass

    for i in range(3):
        p.terminate()
        try:
            await asyncio.wait_for(p.wait(), 10)
            logger.info(f"terminate_asyncio_process(): Process {p.pid} terminated.")
            return
        except asyncio.TimeoutError:
            logger.warning(
                f"terminate_asyncio_process(): Process {p.pid} not terminated, retrying..."
            )

    logger.warning(f"Killing the process {p.pid}...")
    p.kill()
    await p.wait()
    logger.warning(f"terminate_asyncio_process(): Process {p.pid} killed!")

In [ ]:
listener_port = 9099
with generate_app_in_tmp() as app:
    async with ApacheKafkaBroker(listener_port=listener_port) as bootstrap_server:
        os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(
            ":"
        )
        proc = await asyncio.create_subprocess_exec(
            "run_fastkafka_server_process",
            "--kafka-broker",
            "localhost",
            app,
            stdout=asyncio.subprocess.PIPE,
        )
        sleep(5)
        await terminate_asyncio_process(proc)
        outputs, _ = await proc.communicate()

        print(outputs.decode("utf-8"))
        assert proc.returncode == 0

23-05-12 13:15:47.644 [INFO] fastkafka._components.test_dependencies: Java is already installed.
23-05-12 13:15:47.645 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-05-12 13:15:47.646 [INFO] fastkafka._components.test_dependencies: Kafka is installed.
23-05-12 13:15:47.647 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-05-12 13:15:47.647 [INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
23-05-12 13:15:48.375 [INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
23-05-12 13:15:50.279 [INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9099
23-05-12 13:15:55.286 [INFO] __main__: terminate_asyncio_process(): Terminating the process 3741...
23-05-12 13:15:56.446 [INFO] __main__: terminate_asyncio_process(): Process 3741 terminated.
23-05-12 13:15:50.735 [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers

23-05-12 13:15:58.201 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 3176 terminated.
23-05-12 13:15:58.201 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 2797...
23-05-12 13:15:59.550 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 2797 terminated.


In [ ]:
# | export


async def run_fastkafka_server(num_workers: int, app: str, kafka_broker: str) -> None:
    loop = asyncio.get_event_loop()

    HANDLED_SIGNALS = (
        signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
        signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
    )

    d = {"should_exit": False}

    def handle_exit(sig: int, d: Dict[str, bool] = d) -> None:
        d["should_exit"] = True

    for sig in HANDLED_SIGNALS:
        loop.add_signal_handler(sig, handle_exit, sig)

    async with asyncer.create_task_group() as tg:
        args = [
            "run_fastkafka_server_process",
            "--kafka-broker",
            kafka_broker,
            app,
        ]
        tasks = [
            tg.soonify(asyncio.create_subprocess_exec)(
                *args,
                stdout=asyncio.subprocess.PIPE,
                stdin=asyncio.subprocess.PIPE,
            )
            for i in range(num_workers)
        ]

    procs = [task.value for task in tasks]

    async def log_output(
        output: Optional[asyncio.StreamReader], pid: int, d: Dict[str, bool] = d
    ) -> None:
        if output is None:
            raise RuntimeError("Expected StreamReader, got None. Is stdout piped?")
        while not output.at_eof():
            outs = await output.readline()
            if outs != b"":
                typer.echo(f"[{pid:03d}]: " + outs.decode("utf-8"), nl=False)

    async with asyncer.create_task_group() as tg:
        for proc in procs:
            tg.soonify(log_output)(proc.stdout, proc.pid)

        while not d["should_exit"]:
            await asyncio.sleep(0.2)

        typer.echo("Starting process cleanup, this may take a few seconds...")
        for proc in procs:
            tg.soonify(terminate_asyncio_process)(proc)

    for proc in procs:
        output, _ = await proc.communicate()
        if output:
            typer.echo(f"[{proc.pid:03d}]: " + output.decode("utf-8"), nl=False)

    returncodes = [proc.returncode for proc in procs]
    if not returncodes == [0] * len(procs):
        typer.secho(
            f"Return codes are not all zero: {returncodes}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(1)

In [ ]:
# | export


@contextmanager
def run_in_process(
    target: Callable[..., Any]
) -> Generator[multiprocessing.Process, None, None]:
    p = multiprocessing.Process(target=target)
    try:
        p.start()
        yield p
    except Exception as e:
        print(f"Exception raised {e=}")
    finally:
        p.terminate()
        p.join()

In [ ]:
# | notest

listener_port = 10000
async with ApacheKafkaBroker(listener_port=listener_port) as bootstrap_server:
    os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(":")

    with generate_app_in_tmp() as app:

        def run_fastkafka_server_test():
            asyncio.run(run_fastkafka_server(4, app, "localhost"))

        with run_in_process(run_fastkafka_server_test) as p:
            sleep(15)

        assert p.exitcode == 0, p.exitcode
        p.close()

print("ok")

23-05-12 13:15:59.593 [INFO] fastkafka._components.test_dependencies: Java is already installed.
23-05-12 13:15:59.594 [INFO] fastkafka._components.test_dependencies: Kafka is installed.
23-05-12 13:15:59.595 [INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
23-05-12 13:16:00.327 [INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
23-05-12 13:16:02.192 [INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:10000
[4782]: 23-05-12 13:16:02.847 [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to '127.0.0.1:10000'
[4782]: 23-05-12 13:16:02.847 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:10000'}'
[4780]: 23-05-12 13:16:02.851 [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to '127.0.0.1:10000'
[4780]: 23-05-12 13:16:02.851 [INFO] fastkafka._application.app: _creat

[4784]: 23-05-12 13:16:02.900 [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'realitime_data'}
[4784]: 23-05-12 13:16:02.900 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[4780]: 23-05-12 13:16:02.900 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[4780]: 23-05-12 13:16:02.900 [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'realitime_data'})
[4780]: 23-05-12 13:16:02.900 [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'realitime_data'}
[4780]: 23-05-12 13:16:02.901 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[4784]: 23-05-12 13:16:02.905 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[4784]: 23-05-12 13:16:02.905 [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'training_data'})

[4786]: 23-05-12 13:16:03.126 [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[4780]: 23-05-12 13:16:03.168 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[4780]: 23-05-12 13:16:03.168 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[4780]: 23-05-12 13:16:03.171 [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[4780]: 23-05-12 13:16:03.172 [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[4786]: 23-05-12 13:16:03.173 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[4786]: 23-05-12 13:16:03.177 [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[4782]: 23-05-12 13:16:03.228 [ERRO

[4784]: 23-05-12 13:16:03.542 [ERROR] aiokafka.cluster: Topic training_data not found in cluster metadata
[4786]: 23-05-12 13:16:03.542 [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[4782]: 23-05-12 13:16:03.543 [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[4782]: 23-05-12 13:16:03.544 [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[4780]: 23-05-12 13:16:03.588 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[4780]: 23-05-12 13:16:03.589 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[4780]: 23-05-12 13:16:03.590 [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[4780]: 23-05-12 13:16:03.591 [WARNING] aiokafka.cluster: Topic realitime_data 

[4786]: 23-05-12 13:16:03.935 [INFO] aiokafka.consumer.group_coordinator: Joined group '127.0.0.1:10000_group' (generation 1) with member_id aiokafka-0.8.0-d700f6b6-54d6-4f4a-8914-c7f97654cf5c
[4780]: 23-05-12 13:16:03.937 [INFO] aiokafka.consumer.group_coordinator: Joined group '127.0.0.1:10000_group' (generation 1) with member_id aiokafka-0.8.0-b7229ef6-d1b0-493e-a8de-e0a27e3a46c4
[4780]: 23-05-12 13:16:03.937 [INFO] aiokafka.consumer.group_coordinator: Elected group leader -- performing partition assignments using roundrobin
[4780]: 23-05-12 13:16:03.939 [INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'training_data': 1, 'realitime_data': 1}. 
[4782]: 23-05-12 13:16:03.959 [INFO] aiokafka.consumer.group_coordinator: Discovered coordinator 0 for group 127.0.0.1:10000_group
[4782]: 23-05-12 13:16:03.959 [INFO] aiokafka.consumer.group_coordinator: Revoking previously assigned partitions set() for group 127.0.0.1:10000_group
[4782]: 23-05-12 13:16:

[4784]: 23-05-12 13:16:07.019 [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1:10000_group with generation 2
[4784]: 23-05-12 13:16:07.020 [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions set() for group 127.0.0.1:10000_group
[4786]: 23-05-12 13:16:07.020 [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1:10000_group with generation 2
[4786]: 23-05-12 13:16:07.020 [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions set() for group 127.0.0.1:10000_group
[4786]: 23-05-12 13:16:07.021 [INFO] aiokafka.consumer.group_coordinator: Successfully synced group 127.0.0.1:10000_group with generation 2
[4786]: 23-05-12 13:16:07.021 [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions set() for group 127.0.0.1:10000_group
Starting process cleanup, this may take a few seconds...
23-05-12 13:16:17.469 [INFO] __main__: terminate_asyncio_process(): Terminating the p